In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [4]:
def df_with_obj(df, n):
    return df[df['obj_init'] == n]

def is_successful(df, thr=30):
    return df.shape[0] < thr

def success_rate(df, thr=30):
    suc = df[['run']].groupby(['run']).apply(lambda x: is_successful(x, thr)).sum()
    total = pd.unique(df['run']).shape[0]
    
    return suc / total

def avg_successful_pushes(df):
    cnt = df.groupby(['run']).size().reset_index(name='counts')
    cnt = cnt['counts'].sum()
    
    total = pd.unique(df['run']).shape[0]
    
    return cnt / total

def sum_mask_rewards(df):
    sm = df.groupby(['run'])    
    sm = sm['diff_seg_reward'].sum()
    sm = sm.sum()
    
    total = pd.unique(df['run']).shape[0]

    return sm / total

def success_rate_on_run_intervals(df, success_thr, interval):
    df['run_interval'] = df['run'] // interval
    temp = df.groupby(df['run_interval']).apply(lambda x: x.groupby(df['run']).apply(lambda x: is_successful(x, success_thr)).mean())
    temp = temp.to_frame(name='success_rate').reset_index()
    
    return temp 

def target_found_per_session(df):
    return df.loc[df['session'] != 0].groupby(df['session']).mean()['obj_found']

def target_init_per_session(df):
    return df.loc[df['session'] != 0].groupby(df['session']).mean()['obj_init']

def reward_diff_on_run_intervals(df, interval):
    df['run_interval'] = df['run'] // interval
    temp = df.groupby(df['run_interval']).apply(lambda x: x.groupby(df['run']).apply(lambda x: x['diff_seg_reward']).sum())

    return temp

def loss_on_run_intervals(df, interval):
    df['run_interval'] = df['run'] // interval
    temp = df.groupby(df['run_interval']).apply(lambda x: x.groupby(df['run']).apply(lambda x: x['loss']).mean())

    return temp

def plot_success_rate_on_run_intervals(df, success_thrs, interval):
    dfs = []

    df['run_interval'] = df['run'] // interval


    labels = pd.unique(df['run_interval'])

    for t in success_thrs: 
        dfs.append(success_rate_on_run_intervals(df, t, interval))

    fig, ax = plt.subplots()

    for i, temp in enumerate(dfs):
        ax.bar(pd.unique(df['run_interval']), temp['success_rate'] * 100, label=f'{success_thrs[i]} pushes')

    ax.set_xlabel('session group')
    ax.set_ylabel('success rate (%)')
    
    ax.set_title(f'Training success rates for different push limits\ngroups of {interval} consecutive sessions')
    
    ax.set_xticks(pd.unique(df['run_interval']))
    
    ax.legend()
    
def plot_targets_found_per_session_iteration(df):
    found = target_found_per_session(df)

    fig, ax = plt.subplots()

    ax.bar(found.index, found)
    ax.hlines(target_init_avg(df), found.index[0], found.index[-1], linestyles='dashed', colors='gray', label='average initial targets')

    ax.set_xlabel('session iteration')
    ax.set_ylabel('visible targets')
    
    ax.set_title(f'Average number of visible targets')
    
    ax.set_xticks(list(range(1, found.index[-1] + 1)))
    ax.set_yticks(list(range(5)))
    
    ax.legend()
    
# max 3 thresholds!
def plot_success_rate_on_run_intervals_group_bars(df, success_thrs, interval):
    
    dfs = []

    df['run_interval'] = df['run'] // interval

    labels = pd.unique(df['run_interval'])
    x = np.arange(len(labels))

    for t in success_thrs: 
        dfs.append(success_rate_on_run_intervals(df, t, interval))
        
    width = 0.3

    fig, ax = plt.subplots()

    rects = []
    
    rects.append(ax.bar(x - width, dfs[0]['success_rate'] * 100, width, label=f'{success_thrs[0]} pushes'))
    rects.append(ax.bar(x, dfs[1]['success_rate'] * 100, width, label=f'{success_thrs[1]} pushes'))
    rects.append(ax.bar(x + width, dfs[2]['success_rate'] * 100, width, label=f'{success_thrs[2]} pushes'))

    ax.set_xlabel('session interval')
    ax.set_ylabel('success rate (%)')
    
    ax.set_title(f'Training success rates for different push limits\ngroups of {interval} consecutive sessions')
    
    ax.set_xticks(labels)
    
    ax.legend()
    
def avg_targets_found_on_run_intervals(df, interval):
    df['run_interval'] = df['run'] // interval
    temp = df.groupby(df['run_interval']).apply(lambda x: x.groupby(df['run']).apply(lambda x: x['obj_found']).mean())
    
    return temp 
    
def plot_found_targes_on_run_intervals(df, interval):
    
    found = avg_targets_found_on_run_intervals(df, interval)

    labels = found.index

    fig, ax = plt.subplots()

    ax.bar(labels, found)
    ax.hlines(target_init_avg(df), found.index[0], found.index[-1], linestyles='dashed', colors='gray', label='average initial targets')

    ax.set_xlabel('session group')
    ax.set_ylabel('found targets')
    
    ax.set_title(f'Average number of found targets during training\ngroups of {interval} consecutive sessions')
    
    ax.set_xticks(found.index)
    
    ax.legend()
    
    
def target_init_avg(df):
    return df.groupby(df['run']).apply(lambda x: x.groupby(df['obj_init']).mean()).mean()['obj_init']